#  설계

## 클래스

### Crawler

웹툰 크롤러에 대한 모든 기능을 가지고 있는 클래스

#### 클래스 속성

- 웹툰 목록 URL

#### 프로퍼티

- html (웹툰 목록 페이지의 HTML을 가져옴)
- webtoon_data_list (WebtoonData목록을 가져옴)

#### 메서드




### WebtoonData
웹툰 하나에 해당

#### 속성

-- 기존보기 --

- 웹툰 고유 ID (webtoon_id)
- 제목 (title)
- 커버이미지 (url_img_thumbnail)
- (protected)작가 (\_author)
- (protected)작품설명 (\_description)

#### 프로퍼티

-- 상세보기 --
- 작가 (author) 상세보기에 들어가서 author 속성을 채우기
- 작품설명 (description) 상세보기에 들어가서 description 속성을 채우기

#### 메서드

### EpisodeData
웹툰의 한 회에 해당

#### 속성

- 에피소드 고유 아이디 (episode_id)
- 제목 (title)
- 커버이미지 (url_img_thumbnail)
- 별점 (rating)
- 등록일 (created_date)

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
response = requests.get('https://comic.naver.com/webtoon/weekday.nhn')
open('weekday.html', 'wt').write(response.text)

243637

In [5]:
html = open('weekday.html', 'rt').read()
soup = BeautifulSoup(html, 'lxml')

In [38]:
# div class="list_area daily_all" 요소가 가진
# 모든 .col 목록들을 col_list에 할당

# .list_area.daily_all은 웹툰 전체 부분에 해당
# .col요소는 각 요일 부분에 해당

col_list = soup.select_one('div.list_area.daily_all').select('.col')

In [42]:
# .col 내부에 있는 li 요소 한개가 웹툰 한개에 해당
# .col_list(요일 목록)을 순회하며 각 col(요일)이 가진 모든 웹툰(li)요소들을
# li_list에 추가하기

li_list = [
    li
        for col in col_list
            for li in col.select('.col_inner ul > li')
]

In [53]:
len(li_list)

220

In [58]:
class WebtoonData:
    def __init__(self, weboon_id, title, url_thumbnail):
        self.webtoon_id = webtoon_id
        self.title = title
        self.url_thumbnail = url_thumbnail
        
    def __repr__(self):
        return self.title

In [72]:
import re

webtoon_data_dict = {}
#webtoon_data_list = []

for li in li_list:
    title = li.select_one('a.title').get_text(strip=True)
    url_thumbnail = li.select_one('.thumb > a > img')['src']
    webtoon_id = re.match(r'.*titleId=(\d+).*', li.select_one('.thumb > a')['href']).group(1)
    
    if not title in webtoon_data_dict:
        new_webtoon_data = WebtoonData(webtoon_id, title, url_thumbnail)
        #webtoon_data_list.append(new_webtoon_data)
        # 중복 할당을 하는 문제가 발생한다. 해결책으로 KEY가 존재하는지 in 연산자로 찾게한다.
        webtoon_data_dict[title] = new_webtoon_data

In [73]:
print('Dict: ', len(webtoon_data_dict))
print('List: ', len(webtoon_data_list))

Dict:  192
List:  220


In [85]:
url = 'https://comic.naver.com/webtoon/list.nhn?titleId=183559'
response = requests.get(url)
html = response.text

In [86]:
soup = BeautifulSoup(html, 'lxml')

In [87]:
# 에피소드 고유 아이디 (episode_id)
#제목 (title)
#커버이미지 (url_img_thumbnail)
#별점 (rating)
#등록일 (created_date)

In [93]:
episode_list = list(filter(lambda e: e.get('class') == None, soup.select('div#content > table.viewList tr')[1:]))

print(episode_list)

[<tr>
<td>
<a href="/webtoon/detail.nhn?titleId=183559&amp;no=403&amp;weekday=mon" onclick="clickcr(this,'lst.img','183559','403',event)">
<img alt="2부 321화" height="41" onerror="this.src='https://static-comic.pstatic.net/staticImages/COMICWEB/NAVER/img/common/non71_41.gif'" src="https://shared-comic.pstatic.net/thumb/webtoon/183559/403/thumbnail_202x120_ee970c1f-eea6-4517-aef6-797057fa3584.jpg" title="2부 321화" width="71"/>
<span class="mask"></span>
</a>
</td>
<td class="title">
<a href="/webtoon/detail.nhn?titleId=183559&amp;no=403&amp;weekday=mon" onclick="clickcr(this,'lst.title','183559','403',event)">2부 321화</a>
</td>
<td>
<div class="rating_type">
<span class="star"><em style="width:99.31%">평점</em></span>
<strong>9.93</strong>
</div>
</td>
<td class="num">2018.09.30</td>
</tr>, <tr>
<td>
<a href="/webtoon/detail.nhn?titleId=183559&amp;no=402&amp;weekday=mon" onclick="clickcr(this,'lst.img','183559','402',event)">
<img alt="2부 320화" height="41" onerror="this.src='https://static-c

In [94]:
import re

for one_episode in episode_list:
    episode_id = re.search(r'no=(\d+)', one_episode.select('td')[0].select_one('a')['href']).group(1)
    title = one_episode.select_one('td.title > a').get_text()
    url_img_thumbnail = one_episode('td')[0].select_one('a > img')['src']
    rating = one_episode.select_one('td div.rating_type strong').get_text()
    created_date = one_episode.select_one('td.num').get_text()

    print(episode_id, title, rating, created_date, url_img_thumbnail)

403 2부 321화 9.93 2018.09.30 https://shared-comic.pstatic.net/thumb/webtoon/183559/403/thumbnail_202x120_ee970c1f-eea6-4517-aef6-797057fa3584.jpg
402 2부 320화 9.97 2018.09.23 https://shared-comic.pstatic.net/thumb/webtoon/183559/402/thumbnail_202x120_b1426349-ae94-4eaf-bd63-9b2af4bdef2c.jpg
401 2부 319화 9.98 2018.09.16 https://shared-comic.pstatic.net/thumb/webtoon/183559/401/thumbnail_202x120_f851ac1e-02ab-430c-84df-abf304aafc83.jpg
400 2부 318화 9.96 2018.09.09 https://shared-comic.pstatic.net/thumb/webtoon/183559/400/thumbnail_202x120_7fe1272a-9451-4870-bc16-24f38361ee4b.jpg
399 2부 317화 9.97 2018.09.02 https://shared-comic.pstatic.net/thumb/webtoon/183559/399/thumbnail_202x120_16e00f43-8310-4e29-a638-4c3281275192.jpg
398 2부 316화 9.97 2018.08.26 https://shared-comic.pstatic.net/thumb/webtoon/183559/398/thumbnail_202x120_d3777f52-76c7-4e0c-9fdb-4842893598a0.jpg
397 2부 315화 9.96 2018.08.19 https://shared-comic.pstatic.net/thumb/webtoon/183559/397/thumbnail_202x120_71bbf3a3-1c89-414b-9010-4d